In [1]:
from vtk import vtkUnstructuredGridReader
from vtk import vtkXMLUnstructuredGridReader
from vtk.util import numpy_support as VN
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# A python code that plots the maximum flexure of the beam at each output time. This code iterates
# over every .vtu file output by ASPECT, so it may take a little time to run.
# The arguments of the function are:
#    sol_dir_path - the pathway to the solutions directory which contains all of the .vtu files
#    beam_name - the name of the composition field in the input file that makes up the beam
#    beambot - the y-value that is set in the input file as the base of the beam
#######################################################################################
def flex_time(sol_dir_path, beam_name, beambot):
    solutions = os.listdir( sol_dir_path ) # list all files in the solution directory
    minvals = []
    # iterate over all .vtu files in the solutions directory
    for j, soln in enumerate(solutions):
        if j%3==0: # take every 3rd file, since there is a .vtu, .pvtu, and .visit file created 
                   # for each  output, but only the .vtu file is needed
                
            filepath = os.path.join(sol_dir_path, soln) # join the .vtu file to the solution directory path
            # read in the data from the .vtu file
            reader = vtkXMLUnstructuredGridReader()
            reader.SetFileName(filepath)
            reader.Update()
            data = reader.GetOutput()
            points = data.GetPoints()
            x = vtk_to_numpy(points.GetData())
            comp = vtk_to_numpy(data.GetPointData().GetArray(beam_name))
            
            # create an array that only contains the points where the beam is located
            beam = []
            for n in range(len(comp)):
                if comp[n] > 0.5:
                    beam.append(x[n])
            beam = np.array(beam)
            minvals.append(min(beam[:, 1])) # append minimum y value of beam to minvals array
    
    # Plot
    output_num = np.arange(0, len(minvals))
    plt.figure(dpi = 140)
    plt.scatter(output_num, (np.array(minvals) - beambot) / 1e3, s = 5)
    plt.xlabel('Output Number')
    plt.ylabel('Deflection - km')
    plt.title('Maximum Flexure vs Time')